In [27]:
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")


In [28]:
!pip install -q langchain-google-genai google-generativeai langchain-core requests

In [29]:
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [30]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [31]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [32]:
multiply.name

'multiply'

In [33]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [34]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [35]:
# tool binding

In [36]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')


In [37]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--60a4bcf4-f2eb-4daa-af7d-d4191140d5dd-0', usage_metadata={'input_tokens': 2, 'output_tokens': 10, 'total_tokens': 350, 'input_token_details': {'cache_read': 0}})

In [38]:
llm_with_tools = llm.bind_tools([multiply])

In [39]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content='I am an AI assistant, I do not have feelings. How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--7b00792e-465c-43c6-a922-6d20917a970e-0', usage_metadata={'input_tokens': 58, 'output_tokens': 19, 'total_tokens': 77, 'input_token_details': {'cache_read': 0}})

In [40]:
query = HumanMessage('can you multiply 3 with 1000')

In [41]:
messages = [query]

In [42]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [43]:
result = llm_with_tools.invoke(messages)

In [44]:
messages.append(result)

In [45]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--be0ddca3-1697-43d3-91c7-73e05c631f82-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '778be738-b756-4152-b075-b107220f5de9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}})]

In [46]:
tool_result = multiply.invoke(result.tool_calls[0])

In [47]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='778be738-b756-4152-b075-b107220f5de9')

In [48]:
messages.append(tool_result)

In [49]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--be0ddca3-1697-43d3-91c7-73e05c631f82-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': '778be738-b756-4152-b075-b107220f5de9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 65, 'output_tokens': 21, 'total_tokens': 162, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='778be738-b756-4152-b075-b107220f5de9')]

In [50]:
llm_with_tools.invoke(messages).content

'The product of 3 and 1000 is 3000.'

In [68]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  # url = f'https://v6.exchangerate-api.com/v6/{your_api_key}/pair/{base_currency}/{target_currency}'
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [69]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [70]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1763078401,
 'time_last_update_utc': 'Fri, 14 Nov 2025 00:00:01 +0000',
 'time_next_update_unix': 1763164801,
 'time_next_update_utc': 'Sat, 15 Nov 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7501}

In [71]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [72]:
# tool binding
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')


In [73]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [74]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [75]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [76]:
ai_message = llm_with_tools.invoke(messages)

In [77]:
messages.append(ai_message)

In [78]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'USD', 'base_currency': 'INR'},
  'id': '44da3cd6-4a56-487e-975b-1b561acaac1b',
  'type': 'tool_call'}]

In [63]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [64]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_factor', 'arguments': '{"target_currency": "USD", "base_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--88318b25-675f-4aef-90b1-2b358ce8c261-0', tool_calls=[{'name': 'get_conversion_factor', 'args': {'target_currency': 'USD', 'base_currency': 'INR'}, 'id': 'c1fb6601-179c-41f6-ba47-d52fbbeb66ba', 'type': 'tool_call'}], usage_metadata={'input_tokens': 150, 'output_tokens': 26, 'total_tokens': 593, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1763078

In [65]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.01127.\nBased on that, 10 INR is equal to 0.1127 USD.'

In [66]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

/tmp/ipython-input-627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [67]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought: I don't need to use any tools to respond to a greeting. I can directly answer the user.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I am doing well, thank you for asking! How can I assist you today?"
}
```

> Finished chain.
